## ⚽ Football ETL – Transfermarkt ➜ Spark ➜ MongoDB + Neo4j
###  João Vitor Averaldo Antunes (813979) · Pedro Enrico Barchi Nogueira (813099) · Rafael Mori Pinheiro (813851)

UFSCar-CC-So-PMD2025-Grupo 10

### Profa. Dra. Sahudy Montenegro González


---
Carregamento de Paths e Dependências

# Configuração do Ambiente de Execução

Este notebook requer uma configuração de ambiente específica para garantir que o Apache Spark, Delta Lake e os conectores de banco de dados funcionem corretamente. 

---

### 1. Requisitos de Sistema

As seguintes tecnologias devem estar instaladas e configuradas na máquina local:

* **Java Development Kit (JDK):** Versão **17 ou superior**.
    * *Motivo:* O Apache Spark executa na JVM (Java Virtual Machine).
    * *Verificação:* A variável de ambiente `JAVA_HOME` deve estar configurada e apontando para o diretório de instalação do JDK.

* **Apache Spark (Binários):** Versão **4.0.0**.
    * *Motivo:* Esta é a engine de processamento massivo de dados utilizada.
    * *Verificação:* A variável de ambiente `SPARK_HOME` deve estar configurada e apontando para o diretório onde o Spark foi descompactado.

* **Python:** Versão **3.11.x**.
    * *Motivo:* As bibliotecas `pyspark` e `delta-spark` utilizadas neste projeto são compatíveis com esta versão do Python. Versões mais recentes (como 3.13+) podem causar erros de incompatibilidade.
    * *Recomendação (macOS/Linux):* Usar o `pyenv` para gerenciar a versão do Python (`pyenv install 3.11` e `pyenv local 3.11` na pasta do projeto).

---

### 2. Ambiente Virtual e Dependências Python

Para isolar as dependências, este projeto deve ser executado dentro de um ambiente virtual (`venv`).

1.  **Criação:** `python -m venv venv`
2.  **Ativação:** `source venv/bin/activate` (macOS/Linux) ou `.\venv\Scripts\activate` (Windows)

As bibliotecas Python necessárias serão instaladas pela célula de código abaixo. As principais são:
* `pyspark`: A API Python para interagir com o Spark.
* `delta-spark`: A biblioteca para habilitar o suporte ao formato Delta Lake.
* `python-dotenv`: Para gerenciar variáveis de ambiente (credenciais) a partir de um arquivo `.env`.

---

### 3. Arquivos de Conectores (.jar)

Devido a possíveis restrições de rede ou para garantir consistência, este projeto utiliza o método de carregamento manual de JARs. Crie uma pasta chamada `jars` na raiz do projeto e certifique-se de que ela contém os seguintes quatro arquivos:

1.  `delta-spark_2.13-4.0.0.jar`
2.  `delta-storage-4.0.0.jar`
3.  `mongo-spark-connector_2.13-10.5.0.jar`
4.  `mongodb-driver-sync-5.5.1.jar`
5.  `mongodb-driver-core-5.1.2.jar`
6.  `neo4j-connector-apache-spark_2.13-5.3.8_for_spark_3.jar`
7.  `bson-5.1.2.jar`

---

### 4. Arquivo de Variáveis de Ambiente (`.env`)

Crie um arquivo chamado `.env` na raiz do projeto para armazenar as credenciais dos bancos de dados de forma segura. O arquivo deve seguir o seguinte formato:

```plaintext
# Credenciais do MongoDB
MONGO_URI="mongodb+srv://<user>:<password>@<cluster-url>/"

# Credenciais do Neo4j
NEO4J_URI="bolt://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="your_password"

In [ ]:
import os
import sys

# Instala os pacotes necessários
!{sys.executable} -m pip install "pyspark>=4.0.0" "delta-spark>=4.0.0" python-dotenv

print("\n✅ Dependências Python instaladas com sucesso!")
print("Lembre-se de reiniciar o kernel ('Kernel > Restart Kernel...') para que as mudanças tenham efeito.")

### Inicialização da Sessão Spark

A célula a seguir inicializa a `SparkSession`, que é o ponto de entrada para qualquer funcionalidade do Spark. Utilizamos o padrão de projeto **Builder** para configurar a sessão antes de criá-la.

As configurações aplicadas são cruciais:
* `appName`: Define um nome para a nossa aplicação, útil para monitoramento na UI do Spark.
* `spark.jars`: Especifica os caminhos para os arquivos `.jar` que carregamos manualmente. Esta abordagem foi escolhida para contornar possíveis problemas de rede e garantir consistência, fornecendo ao Spark os drivers para **Delta Lake**, **MongoDB** e **Neo4j**.
* `spark.sql.extensions` e `spark.sql.catalog.spark_catalog`: Habilitam as funcionalidades avançadas do Delta Lake, como o suporte a comandos SQL específicos do Delta.
* `spark.mongodb.*`: Pré-configura os parâmetros de conexão padrão para o MongoDB, que serão usados nas operações de escrita.
* `spark.neo4j.*`: Pré-configura os parâmetros de conexão padrão para o Neo4j, que serão usados nas operações de escrita.

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession, functions as F, types as T, Window, Row
from dotenv import load_dotenv
import os
from typing import List
from datetime import datetime

# -- variáveis de ambiente --
load_dotenv('./.env')

BASE_PATH = "./raw_data"
PATH      = "./pre-processing"        # onde vai gravar os Delta
os.makedirs(PATH, exist_ok=True)

MONGO_URI  = os.getenv("MONGO_URI")
NEO4J_URL  = os.getenv("NEO4J_URI")
NEO4J_AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

# --- Caminhos para os JARs locais ---
delta_spark_jar     = "jars/delta-spark_2.13-4.0.0.jar"
delta_storage_jar   = "jars/delta-storage-4.0.0.jar"

mongo_connector_jar = "jars/mongo-spark-connector_2.13-10.5.0.jar"
mongo_driver_core   = "jars/mongodb-driver-core-5.1.2.jar"
mongo_driver_sync   = "jars/mongodb-driver-sync-5.1.2.jar"
mongo_bson          = "jars/bson-5.1.2.jar"

# Caminho para o JAR do Neo4j Spark Connector
neo4j_jar = "jars/neo4j-connector-apache-spark_2.13-5.3.8_for_spark_3.jar"

# --- Constrói e CRIA a sessão Spark ---
print("Iniciando a construção da sessão Spark para Delta e MongoDB...")

spark = (
    SparkSession.builder
      .appName("FootballPipeline-MongoFocus")
      .config(
          "spark.jars",
          ",".join([
              delta_spark_jar,
              delta_storage_jar,
              mongo_connector_jar,
              mongo_driver_core,
              mongo_driver_sync,
              mongo_bson,
              neo4j_jar
          ])
      )
      .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
      .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
      .config("spark.mongodb.write.connection.uri", MONGO_URI)   
      .config("spark.mongodb.write.database", "football")
      .config("spark.neo4j.url", NEO4J_URL)  # URL do Neo4j
      .config("spark.neo4j.authentication.basic.username", NEO4J_AUTH[0])  # Username
      .config("spark.neo4j.authentication.basic.password", NEO4J_AUTH[1])  # Password
      .getOrCreate()
)

print(f"✅ Sessão Spark para Delta e MongoDB iniciada com sucesso! Versão: {spark.version}")

Iniciando a construção da sessão Spark para Delta e MongoDB...
✅ Sessão Spark para Delta e MongoDB iniciada com sucesso! Versão: 4.0.0


--- 
### Definição Explícita dos Schemas

Antes de carregar os dados, definimos explicitamente o `schema` (a estrutura e os tipos de dados) para cada um dos nossos arquivos CSV de origem.

Esta é uma **prática recomendada fundamental** em pipelines de dados robustos, em vez de usar a opção `inferSchema=True` do Spark, pelos seguintes motivos:

* **Performance:** Evita que o Spark precise ler os dados uma vez a mais apenas para inferir os tipos de dados, o que pode ser muito custoso em datasets grandes.
* **Integridade e Prevenção de Erros:** Garante que os dados sejam lidos com os tipos corretos desde o início. Isso previne erros de tipagem em operações futuras.
* **Consistência:** O pipeline não quebrará se uma amostra dos dados levar o Spark a inferir um tipo incorreto.

In [2]:
appearances_schema = T.StructType([
    T.StructField("appearance_id",              T.StringType()),
    T.StructField("game_id",                    T.LongType()),
    T.StructField("player_id",                  T.LongType()),
    T.StructField("player_club_id",             T.LongType()),
    T.StructField("player_current_club_id",     T.LongType()),
    T.StructField("date",                       T.DateType()),
    T.StructField("player_name",                T.StringType()),
    T.StructField("competition_id",             T.StringType()),
    T.StructField("yellow_cards",               T.IntegerType()),
    T.StructField("red_cards",                  T.IntegerType()),
    T.StructField("goals",                      T.IntegerType()),
    T.StructField("assists",                    T.IntegerType()),
    T.StructField("minutes_played",             T.IntegerType())
])

In [3]:
club_games_schema = T.StructType([
    T.StructField("game_id",            T.LongType()),
    T.StructField("club_id",            T.LongType()),
    T.StructField("own_goals",          T.IntegerType()),
    T.StructField("own_position",       T.IntegerType()),
    T.StructField("own_manager_name",   T.StringType()),
    T.StructField("opponent_id",        T.LongType()),
    T.StructField("opponent_goals",     T.IntegerType()),
    T.StructField("opponent_position",  T.IntegerType()),
    T.StructField("opponent_manager_name", T.StringType()),
    T.StructField("hosting",            T.StringType()),
    T.StructField("is_win",             T.IntegerType())
])

clubs_schema = T.StructType([
    T.StructField("club_id",                  T.LongType()),
    T.StructField("club_code",                T.StringType()),
    T.StructField("name",                     T.StringType()),
    T.StructField("domestic_competition_id",  T.StringType()),
    T.StructField("total_market_value",       T.StringType()),
    T.StructField("squad_size",               T.IntegerType()),
    T.StructField("average_age",              T.DoubleType()),
    T.StructField("foreigners_number",        T.IntegerType()),
    T.StructField("foreigners_percentage",    T.DoubleType()),
    T.StructField("national_team_players",    T.IntegerType()),
    T.StructField("stadium_name",             T.StringType()),
    T.StructField("stadium_seats",            T.IntegerType()),
    T.StructField("net_transfer_record",      T.StringType()),
    T.StructField("coach_name",               T.StringType()),
    T.StructField("last_season",              T.IntegerType()),
    T.StructField("filename",                 T.StringType()),
    T.StructField("url",                      T.StringType())
])


In [4]:
competitions_schema = T.StructType([
    T.StructField("competition_id",          T.StringType()),
    T.StructField("competition_code",        T.StringType()),
    T.StructField("name",                    T.StringType()),
    T.StructField("sub_type",                T.StringType()),
    T.StructField("type",                    T.StringType()),
    T.StructField("country_id",              T.IntegerType()),
    T.StructField("country_name",            T.StringType()),
    T.StructField("domestic_league_code",    T.StringType()),
    T.StructField("confederation",           T.StringType()),
    T.StructField("url",                     T.StringType()),
    T.StructField("is_major_national_league",T.BooleanType())
])

In [5]:
game_events_schema = T.StructType([
    T.StructField("game_event_id",   T.StringType()),
    T.StructField("date",            T.DateType()),
    T.StructField("game_id",         T.LongType()),
    T.StructField("minute",          T.IntegerType()),
    T.StructField("type",            T.StringType()),
    T.StructField("club_id",         T.LongType()),
    T.StructField("player_id",       T.LongType()),
    T.StructField("description",     T.StringType()),
    T.StructField("player_in_id",    T.LongType()),
    T.StructField("player_assist_id",T.LongType())
])

game_lineups_schema = T.StructType([
    T.StructField("game_lineups_id", T.StringType()),
    T.StructField("date",            T.DateType()),
    T.StructField("game_id",         T.LongType()),
    T.StructField("player_id",       T.LongType()),
    T.StructField("club_id",         T.LongType()),
    T.StructField("player_name",     T.StringType()),
    T.StructField("type",            T.StringType()),
    T.StructField("position",        T.StringType()),
    T.StructField("number",          T.IntegerType()),
    T.StructField("team_captain",    T.IntegerType())
])

games_schema = T.StructType([
    T.StructField("game_id",              T.LongType()),
    T.StructField("competition_id",       T.StringType()),
    T.StructField("season",               T.IntegerType()),
    T.StructField("round",                T.StringType()),
    T.StructField("date",                 T.DateType()),
    T.StructField("home_club_id",         T.LongType()),
    T.StructField("away_club_id",         T.LongType()),
    T.StructField("home_club_goals",      T.IntegerType()),
    T.StructField("away_club_goals",      T.IntegerType()),
    T.StructField("home_club_position",   T.IntegerType()),
    T.StructField("away_club_position",   T.IntegerType()),
    T.StructField("home_club_manager_name", T.StringType()),
    T.StructField("away_club_manager_name", T.StringType()),
    T.StructField("stadium",              T.StringType()),
    T.StructField("attendance",           T.IntegerType()),
    T.StructField("referee",              T.StringType()),
    T.StructField("url",                  T.StringType()),
    T.StructField("home_club_formation",  T.StringType()),
    T.StructField("away_club_formation",  T.StringType()),
    T.StructField("home_club_name",       T.StringType()),
    T.StructField("away_club_name",       T.StringType()),
    T.StructField("aggregate",            T.StringType()),
    T.StructField("competition_type",     T.StringType())
])

In [6]:
valuations_schema = T.StructType([
    T.StructField("player_id",        T.LongType()),
    T.StructField("date",             T.DateType()),
    T.StructField("market_value_in_eur", T.DoubleType()),
    T.StructField("current_club_id",  T.LongType()),
    T.StructField("player_club_domestic_competition_id", T.StringType())
])

In [7]:
players_schema = T.StructType([
    T.StructField("player_id",            T.LongType()),
    T.StructField("first_name",           T.StringType()),
    T.StructField("last_name",            T.StringType()),
    T.StructField("name",                 T.StringType()),
    T.StructField("last_season",          T.IntegerType()),
    T.StructField("current_club_id",      T.LongType()),
    T.StructField("player_code",          T.StringType()),
    T.StructField("country_of_birth",     T.StringType()),
    T.StructField("city_of_birth",        T.StringType()),
    T.StructField("country_of_citizenship",T.StringType()),
    T.StructField("date_of_birth",        T.TimestampType()),
    T.StructField("sub_position",         T.StringType()),
    T.StructField("position",             T.StringType()),
    T.StructField("foot",                 T.StringType()),
    T.StructField("height_in_cm",         T.IntegerType()),
    T.StructField("contract_expiration_date", T.StringType()),
    T.StructField("agent_name",           T.StringType()),
    T.StructField("image_url",            T.StringType()),
    T.StructField("url",                  T.StringType()),
    T.StructField("current_club_domestic_competition_id", T.StringType()),
    T.StructField("current_club_name",    T.StringType()),
    T.StructField("market_value_in_eur",  T.DoubleType()),
    T.StructField("highest_market_value_in_eur", T.DoubleType())
])

transfers_schema = T.StructType([
    T.StructField("player_id",        T.LongType()),
    T.StructField("transfer_date",    T.DateType()),
    T.StructField("transfer_season",  T.StringType()),
    T.StructField("from_club_id",     T.LongType()),
    T.StructField("to_club_id",       T.LongType()),
    T.StructField("from_club_name",   T.StringType()),
    T.StructField("to_club_name",     T.StringType()),
    T.StructField("transfer_fee",     T.DoubleType()),
    T.StructField("market_value_in_eur", T.DoubleType()),
    T.StructField("player_name",      T.StringType())
])

---
### Camada Bronze: Ingestão dos Dados Brutos

Nesta seção, realizamos a ingestão dos dados. Seguindo os princípios da **arquitetura Medalhão**, esta primeira camada, a **Bronze**, representa os dados em seu estado bruto, exatamente como foram recebidos da fonte.

A função `load_csv` é um utilitário simples que utiliza os `schemas` definidos anteriormente para carregar cada arquivo CSV em um DataFrame do Spark. Cada DataFrame com o sufixo `_bronze` corresponde a uma tabela de dados brutos.


In [8]:
def load_csv(name, schema):
    return (spark.read
            .option("header", True)
            .schema(schema)
            .csv(f"{BASE_PATH}/{name}.csv"))

apps_bronze         = load_csv("appearances",        appearances_schema)
club_games_bronze   = load_csv("club_games",         club_games_schema)
clubs_bronze        = load_csv("clubs",              clubs_schema)
competitions_bronze = load_csv("competitions",       competitions_schema)
events_bronze       = load_csv("game_events",        game_events_schema)
lineups_bronze      = load_csv("game_lineups",       game_lineups_schema)
games_bronze        = load_csv("games",              games_schema)
valuations_bronze   = load_csv("player_valuations",  valuations_schema)
players_bronze      = load_csv("players",            players_schema)
transfers_bronze    = load_csv("transfers",          transfers_schema)

--- 
### Camada Silver: Limpeza, Padronização e Persistência

A camada **Silver** é onde os dados da camada Bronze são transformados em tabelas limpas, consistentes e prontas para análise. As seguintes operações são realizadas:

* **`base_clean`:** Uma função de limpeza básica que remove duplicatas com base em uma chave primária e adiciona um timestamp de ingestão para rastreabilidade.
* **`parse_euro` (UDF):** Uma função criada para tratar um caso específico de parsing. As colunas de valor de mercado no dataset de clubes vêm em um formato de texto não padronizado (ex: `"+€3.05m"`). Esta UDF utiliza uma expressão regular (regex) para extrair o valor numérico e convertê-lo para um tipo `double` em euros.
* **Persistência em Delta:** Após a limpeza, cada DataFrame é salvo no formato **Delta Lake**. Ele materializa os dados limpos no disco, o que melhora a performance das etapas seguintes (que agora lerão de arquivos Parquet otimizados) e garante a recuperabilidade do pipeline em caso de falhas.

In [9]:
def base_clean(df, pk: List[str]):
    return (df
            .dropDuplicates(pk)
            .withColumn("_ingest_timestamp", F.current_timestamp()))

apps_silver = (base_clean(apps_bronze, ["appearance_id"])
               .filter("minutes_played IS NOT NULL"))

# exemplo conversão market value "+€3.05m" → 3.05
@F.udf("double")
def parse_euro(s):
    import re
    if s is None or s == "":
        return None
    m = re.search(r"[-+€]?\s*([\d\.]+)m", s)
    return float(m.group(1))*1e6 if m else None

clubs_silver = (base_clean(clubs_bronze, ["club_id"])
                .withColumn("total_market_value_eur", parse_euro("total_market_value"))
                .withColumn("net_transfer_record_eur", parse_euro("net_transfer_record"))
                .drop("total_market_value","net_transfer_record"))

games_silver = base_clean(games_bronze, ["game_id"])
players_silver = base_clean(players_bronze, ["player_id"])
valuations_silver = base_clean(valuations_bronze, ["player_id","date"])
transfers_silver = base_clean(transfers_bronze, ["player_id","transfer_date"])
club_games_silver = base_clean(club_games_bronze, ["game_id","club_id"])
competitions_silver = base_clean(competitions_bronze, ["competition_id"])
events_silver = base_clean(events_bronze, ["game_event_id"])
lineups_silver = base_clean(lineups_bronze, ["game_lineups_id"])

# Persistir como Delta para reuso
(spark
 .createDataFrame([], T.StructType([])))  # placeholder para evitar saída longa

for name, df in [("apps",apps_silver),("clubs",clubs_silver),("games",games_silver),
                 ("players",players_silver),("valuations",valuations_silver),
                 ("transfers",transfers_silver),("club_games",club_games_silver),
                 ("competitions",competitions_silver),("events",events_silver),
                 ("lineups",lineups_silver)]:
    df.write.mode("overwrite").format("delta").save(f"{PATH}/{name}")

25/07/02 02:12:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/02 02:12:54 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/07/02 02:13:20 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/07/02 02:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/07/02 02:13:37 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/07/02 02:13:51 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/07/02 02:14:06 WARN MemoryManager

### Modelagem para os Eixos de Análise

Com os dados limpos e persistidos na camada Silver, iniciamos a fase de **Engenharia de Atributos** e modelagem. Aqui, aplicamos a lógica de negócio para criar os DataFrames que responderão diretamente às perguntas definidas nos três eixos do projeto.

* **Eixo 1 (Valor de Mercado e ROI):**
    * No DataFrame `valuations_enriched`, utilizamos **funções de janela** (`Window`) para calcular a idade do jogador em cada data de avaliação e a variação percentual de valor (`pct_growth`) em relação à medição anterior.
    * No DataFrame `roi`, unimos dados de transferências e de valorações para calcular um ROI (Retorno sobre Investimento) simplificado, medindo a valorização do atleta nos 12 meses seguintes à sua contratação.

* **Eixo 2 (Impacto Tático):**
    * O DataFrame `apps_enriched` é enriquecido com informações de contexto do jogo (se o jogador estava em casa ou fora, se foi titular ou substituto).
    * `tactical_impact` agrega essas informações por jogador para criar métricas de desempenho médio, como média de gols, assistências e taxa de participação como substituto.

* **Eixo 3 (Carreira e Conexões):**
    * O DataFrame `career_edges` é preparado especificamente para o Neo4j. Ele isola as informações de transferência, formatando os dados para representar as **arestas** (relações `TRANSFERRED_TO`) do nosso futuro grafo de carreiras.

---
Estruturação Eixo 1

In [24]:
w_date = Window.partitionBy("player_id").orderBy("date")

valuations_enriched = (valuations_silver
                       .join(players_silver.select("player_id","date_of_birth"), "player_id")
                       .withColumn("age",
                                   F.floor(F.months_between("date","date_of_birth")/12))
                       .withColumn("prev_value",
                                   F.lag("market_value_in_eur").over(w_date))
                       .withColumn("pct_growth",
                                   (F.col("market_value_in_eur")-F.try_divide(F.col("prev_value"), F.col("prev_value")))))

# ROI de transferências (simples: variação de valor em 12 meses após compra)
buy_side = (transfers_silver
            .select("player_id","transfer_date","to_club_id","transfer_fee")
            .withColumnRenamed("transfer_date","buy_date"))

roi = (buy_side
       .join(valuations_enriched, (valuations_enriched.player_id == buy_side.player_id) &
                                  (valuations_enriched.date >= buy_side.buy_date) &
                                  (valuations_enriched.date <= F.add_months(buy_side.buy_date,12)))
       .groupBy(buy_side.player_id, "to_club_id","transfer_fee")
       .agg(F.max("market_value_in_eur").alias("value_after_12m"))
       .withColumn("roi_pct",
                   (F.col("value_after_12m")-F.try_divide(F.col("transfer_fee"), F.col("transfer_fee")))))

---
Estruturação Eixo 2

In [11]:
apps_enriched = (apps_silver
                 .join(games_silver.select("game_id","home_club_id","away_club_id",
                                           "home_club_goals","away_club_goals"),"game_id")
                 .withColumn("is_home",
                             F.when(F.col("player_club_id")==F.col("home_club_id"),1).otherwise(0))
                 .withColumn("is_sub",
                             F.when((F.col("minutes_played")<90) & (F.col("minutes_played")>0),1).otherwise(0))
                )

tactical_impact = (apps_enriched
                   .groupBy("player_id")
                   .agg(F.avg("goals").alias("avg_goals"),
                        F.avg("assists").alias("avg_assists"),
                        F.avg("is_sub").alias("sub_rate")))

---
Estruturação Eixo 3

In [15]:
career_edges = (transfers_silver
                .select("player_id","from_club_id","to_club_id","transfer_date")
                .withColumnRenamed("transfer_date","date"))

---
### Carga no MongoDB

Nesta etapa, preparamos e salvamos os dados no **MongoDB**. A escolha deste banco de dados se justifica pela sua natureza orientada a documentos, que é ideal para o nosso caso de uso.

O DataFrame `players_doc` é construído como um **documento denormalizado**. Ele agrega, para cada jogador, um conjunto rico de informações:
* Dados pessoais (do `players_silver`).
* Métricas de desempenho tático agregadas (do `tactical_impact`).
* Um **array aninhado** com o histórico completo de suas valorações de mercado.

Essa estrutura de "Perfil 360°" é extremamente eficiente, pois permite recuperar todas as informações de um jogador com uma única consulta, eliminando a necessidade de `JOINs` complexos no momento da leitura, o que é uma grande vantagem do modelo de documentos.

In [20]:
# DataFrame que será usado no join
clubs_join_df = clubs_silver.select(
    F.col("club_id").alias("current_club_id"),
    F.col("name").alias("current_club_name")
)

# Construção do DataFrame 'players_doc' com a correção
players_doc = (players_silver
               .join(tactical_impact, "player_id", "left")
               .join(valuations_enriched.groupBy("player_id")
                     .agg(F.collect_list(F.struct("date", "market_value_in_eur")).alias("valuations")),
                     "player_id", "left")
               # O join que causa a duplicidade
               .join(clubs_join_df, "current_club_id", "left")
               # ✅ A SOLUÇÃO: Remove a coluna duplicada que veio do join com clubs_join_df
               .drop(clubs_join_df.current_club_name)
               .drop("_ingest_timestamp")
)

# Verificação do schema 
print("Schema do DataFrame 'players_doc:")
players_doc.printSchema()

(players_doc
 .write
 .format("mongodb")
 .option("collection","players")
 .mode("overwrite")
 .save())

Schema do DataFrame 'players_doc:
root
 |-- current_club_id: long (nullable = true)
 |-- player_id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- last_season: integer (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- city_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: timestamp (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- contract_expiration_date: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- url: string (nullable = true)
 |-- current_club_domestic_competition_id: string (nullable = true)
 |-- current_club_name: string (nullable = true)
 |-- market_value_in

---
### Carga no Neo4j

Aqui, populamos nosso banco de dados de grafos **Neo4j**. Este modelo é naturalmente adequado para representar e analisar as conexões e trajetórias na carreira dos atletas. O processo é dividido em duas partes:

1.  **Criação dos Nós:**
    * Os `:Player` (nós de jogador) são criados a partir do `players_silver`.
    * Os `:Club` (nós de clube) são criados a partir do `clubs_silver`.
    * Utilizamos a opção `.option("node.keys", "id")` para garantir a **idempotência**, ou seja, que um mesmo jogador ou clube não seja criado mais de uma vez, mesmo que o código seja executado múltiplas vezes.

2.  **Criação das Arestas:**
    * As relações `:TRANSFERRED_TO` são criadas a partir do DataFrame `career_edges`.
    * O conector do Spark mapeia as colunas de origem (`source_id`) e de destino (`target_id`) para conectar os nós `:Player` e `:Club` correspondentes, criando o grafo que representa as transferências.

In [18]:
# Nós: Player
(players_silver
 .selectExpr("player_id as id","name","date_of_birth")
 .write
 .format("org.neo4j.spark.DataSource")
 .option("url", NEO4J_URL)
 .option("authentication.type","basic")
 .option("authentication.basic.username", NEO4J_AUTH[0])
 .option("authentication.basic.password", NEO4J_AUTH[1])
 .option("labels",":Player")
 .option("node.keys", "id")  # Adiciona a chave para identificar o nó
 .mode("overwrite")
 .save())

# Nós: Club
(clubs_silver
 .selectExpr("club_id as id","name")
 .write
 .format("org.neo4j.spark.DataSource")
 .option("url", NEO4J_URL)
 .option("authentication.type","basic")
 .option("authentication.basic.username", NEO4J_AUTH[0])
 .option("authentication.basic.password", NEO4J_AUTH[1])
 .option("labels",":Club")
 .option("node.keys", "id")  # Adiciona a chave para identificar o nó
 .mode("overwrite")
 .save())

# Arestas: TRANSFERRED_TO
(career_edges
 .selectExpr("player_id as source_id",
             "to_club_id as target_id",
             "date")
 .coalesce(1)                                     # evita deadlocks
 .write
 .format("org.neo4j.spark.DataSource")
 .option("url", NEO4J_URL)
 .option("authentication.type","basic")
 .option("authentication.basic.username", NEO4J_AUTH[0])
 .option("authentication.basic.password", NEO4J_AUTH[1])
 .option("relationship","TRANSFERRED_TO")
 .option("relationship.save.strategy","keys")
 .option("relationship.source.labels",":Player")
 .option("relationship.target.labels",":Club")
 .option("relationship.source.node.keys","source_id")
 .option("relationship.target.node.keys","target_id")
 .mode("append")
 .save())


---
### Verificação Final e Métricas de Qualidade

Como etapa final do nosso pipeline ETL, executamos duas ações importantes:

1.  **Cálculo de Métricas de Qualidade:** Para os nossos principais DataFrames enriquecidos, calculamos e persistimos uma tabela de métricas. Isso inclui a contagem total de linhas e um dicionário com a contagem de valores nulos para cada coluna. Esta é uma prática profissional essencial para monitorar a saúde e a integridade dos dados ao longo do tempo.

2.  **Visualização e Validação:** Lemos de volta todas as tabelas Delta que foram salvas na camada Silver e as exibimos com `.show()`. Este passo serve como uma verificação visual final para garantir que o pipeline foi executado como esperado e que os dados estão estruturados corretamente antes de passarmos para a fase de análise e consulta nos bancos de dados NoSQL.

In [28]:
quality_metrics = []

for name, df in [("players_doc", players_doc),
                 ("games_silver", games_silver),
                 ("valuations_enriched", valuations_enriched)]:
    
    print(f"Calculando métricas para a tabela: {name}...")
    
    quality_metrics.append(Row(
        table=name,
        rows=df.count(),
        nulls=df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
              .select([F.sum(c).alias(c) for c in df.columns]).first().asDict(),
        # ✅ PASSO 2: Use a função do Python para pegar o timestamp atual
        timestamp=datetime.now()
    ))

print("\Métricas calculadas. Criando o DataFrame final...")

# Esta parte agora vai funcionar, pois quality_metrics só contém dados Python
(spark.createDataFrame(quality_metrics)
      .write.mode("append").format("delta")
      .save(f"{PATH}/quality_metrics")) # Ajustei o caminho para o seu padrão

print("\n✅ Métricas de qualidade salvas com sucesso!")

Calculando métricas para a tabela: players_doc...


Calculando métricas para a tabela: games_silver...


Calculando métricas para a tabela: valuations_enriched...


\Métricas calculadas. Criando o DataFrame final...



✅ Métricas de qualidade salvas com sucesso!


In [32]:
# Carregando a tabela de métricas de qualidade de volta para um DataFrame
quality_df = spark.read.format("delta").load(f"{PATH}/quality_metrics")
print("Visualizando a tabela de Métricas de Qualidade:")
quality_df.show(truncate=False)

# Carregando a tabela de jogadores
players_final_df = spark.read.format("delta").load(f"{PATH}/players")
print("\nVisualizando a tabela final de Jogadores:")
players_final_df.show(5)

# Carregando a tabela de apariceos
apps_final_df = spark.read.format("delta").load(f"{PATH}/apps")
print("\nVisualizando a tabela final de Aparições:")
apps_final_df.show(5)

# Carregando a tabela de clubes
clubs_final_df = spark.read.format("delta").load(f"{PATH}/clubs")
print("\nVisualizando a tabela final de Clubes:")
clubs_final_df.show(5)

# Carregando a tabela de jogos do clube
club_games_final_df = spark.read.format("delta").load(f"{PATH}/club_games")
print("\nVisualizando a tabela final de Jogos dos clubes:")
club_games_final_df.show(5)

# Carregando a tabela de competicoes
competitions_final_df = spark.read.format("delta").load(f"{PATH}/competitions")
print("\nVisualizando a tabela final de Competições:")
competitions_final_df.show(5)

# Carregando a tabela de eventos
events_final_df = spark.read.format("delta").load(f"{PATH}/events")
print("\nVisualizando a tabela final de Eventos:")
events_final_df.show(5)

# Carregando a tabela de escalações
lineups_final_df = spark.read.format("delta").load(f"{PATH}/lineups")
print("\nVisualizando a tabela final de Escalações:")
lineups_final_df.show(5)

# Carregando a tabela de transferencias
transfers_final_df = spark.read.format("delta").load(f"{PATH}/transfers")
print("\nVisualizando a tabela final de Transferências:")
transfers_final_df.show(5)

# Carregando a tabela de valuations
valuations_final_df = spark.read.format("delta").load(f"{PATH}/valuations")
print("\nVisualizando a tabela final de Valuations:")
valuations_final_df.show(5)

Visualizando a tabela de Métricas de Qualidade:
+-------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|table              |rows  |nulls                                                                                                                                                                                                                                                                                   

+-------------+-------+---------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+--------------------+
|appearance_id|game_id|player_id|player_club_id|player_current_club_id|      date|         player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|   _ingest_timestamp|
+-------------+-------+---------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+--------------------+
| 2218677_2865|2218677|     2865|           506|                   167|2012-08-11|Stephan Lichtsteiner|           SCI|           1|        0|    0|      0|            89|2025-07-02 02:12:...|
|2218677_37666|2218677|    37666|           506|                    46|2012-08-11|        Arturo Vidal|           SCI|           0|        0|    1|      1|           120|2025-07-02 02:12:...|
| 2218677_5880|2218677|     5880|       